# Voice

I'm shelving this until OpenAI exposes their melodic poet voice.

In [1]:
import json

import pandas as pd
import numpy as np
from IPython.display import display

from jsonschema import validate
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

import os, dotenv, sys
from pathlib import Path

dotenv.load_dotenv("../.env", override=True)
print(f"{os.environ["VIRTUAL_ENV"]=}")
print(f"{os.environ["PYTHONSTARTUP"]=}")
parent = Path().resolve().parents[0].as_posix()
if parent not in sys.path:
    sys.path.insert(0, parent)

os.environ["VIRTUAL_ENV"]='/Users/nima/.local/share/virtualenvs/worldly-6Xoo3lh0'
os.environ["PYTHONSTARTUP"]='.repl.py'


In [2]:
import asyncio

import openai, whisper
import azure.cognitiveservices.speech as speechsdk
from openai import ChatCompletion

import speech_recognition as sr
import sqlite3
from langdetect import detect
from playsound import playsound
from gtts import gTTS
from playsound import playsound

# Load Whisper model (choose "base" for faster or "small" for better accuracy)
model = whisper.load_model("base")
client = openai.OpenAI()

/Users/nima/.local/share/virtualenvs/worldly-6Xoo3lh0/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.

In [3]:
# Database Setup
conn = sqlite3.connect("words.db")
cursor = conn.cursor()
cursor.execute(
    """CREATE TABLE IF NOT EXISTS words (word TEXT, language TEXT, correct_count INTEGER, wrong_count INTEGER)"""
)

In [4]:
def detect_language(text):
    prompt = f"""You are a language expert who only detects languages from the following list: Sanskrit, Persian, and Latin. 
                 If the text matches one of these, respond with the language name (e.g., "Latin"). 
                 Otherwise, respond with "None". 
                 Here is the text: "{text}"."""
    response = client.chat.completions.create(model="gpt-4", messages=[{"role": "user", "content": prompt}])
    return response.choices[0].message.content.strip()


# Example usage
text = "Salve, quid agis?"
detected_lang = detect_language(text)
print(f"Detected language: {detected_lang}")

"""_summary_
async def detect_language(text):
    prompt = f""You are a language expert who detects whether text is in Sanskrit, Persian, or Latin. 
    If the text matches one of these, respond with the language name (e.g., "Latin"). 
    If it doesn't match, respond with "None". 
    Here is the text: "{text}".""
    
    response = await openai.ChatCompletion.acreate(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a language detection assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response['choices'][0]['message']['content'].strip()

# Example usage
async def main():
    text = "Salve, quid agis?"
    detected_lang = await detect_language(text)
    print(f"Detected language: {detected_lang}")

asyncio.run(main())
"""
None

Detected language: Latin


In [5]:
def generate_persian_text(prompt):
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert in Persian poetry and literature."},
            {"role": "user", "content": prompt},
        ],
    )
    return response


persian_text = generate_persian_text("Provide a famous verse from the Shahnameh, in Persian.")
print(f"Persian Verse: {persian_text.choices[0].message.content}")

Persian Verse: "برخاست و زین برگه زد برنگ زری
بفرمود تا بیاورندش خری"
-Shahnameh, Ferdowsi


In [12]:
import openai
import requests
import os
from playsound import playsound

openai.api_key = os.getenv("OPENAI_API_KEY")


# Function to generate text using OpenAI
def generate_text(prompt, language="Persian"):
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": f"You are a literature expert in {language}."},
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content


# Function to generate audio using OpenAI TTS
def text_to_speech_openai(text, filename="output.mp3", voice="alloy"):
    url = "https://api.openai.com/v1/audio/speech"
    headers = {"Authorization": f"Bearer {openai.api_key}", "Content-Type": "application/json"}
    data = {"model": "tts-1", "input": text, "voice": voice}

    # Generate audio
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        with open(filename, "wb") as audio_file:
            audio_file.write(response.content)
        print(f"Audio saved to {filename}")
        playsound(filename)
    else:
        print(f"Error: {response.status_code}, {response.text}")


# Main function to generate and play Persian poetry
def main():
    # Step 1: Generate Persian text
    prompt = """Provide a famous verse from the Shahnameh in Persia, in its original Persian.
    Remember, I am speaking to you in Persian, and you are a Persian, without any non-Persian accent.
    Also, please make to use the zebar, zir and piš, and so on, so that the text-to-speech will
    not lack this data.  Your response should be very lean, only with the quoted verse from the
    poem of your choice.
    """
    persian_text = generate_text(prompt, language="Persian")
    print(f"Generated Persian Text: {persian_text}")

    # Step 2: Convert to speech and play
    for voice in ["fable", "onyx", "nova"]:
        text_to_speech_openai(persian_text, filename="persian_poetry.mp3", voice=voice)


if __name__ == "__main__":
    main()

Generated Persian Text: "به یاد کسی مرد نیکوکار نیست. هر آنکس که او با مردم ناخوش گذر کند."
Audio saved to persian_poetry.mp3
Audio saved to persian_poetry.mp3
Audio saved to persian_poetry.mp3
Audio saved to persian_poetry.mp3
Audio saved to persian_poetry.mp3
Audio saved to persian_poetry.mp3


You’re absolutely right — when interacting with OpenAI’s text-to-speech tools or other systems, the richness, emotion, and flow of poetry are key to creating an engaging experience. The voice you get from OpenAI’s current text-to-speech API (tts-1) is high quality but not tailored for expressive readings like poetry.

In ChatGPT’s spoken responses (like on mobile), OpenAI likely uses a different pipeline or a more advanced model to add expressive intonation. Unfortunately, the public TTS models don’t yet include this richer style.

Since OpenAI already demonstrates expressive intonation in its ChatGPT voice features (like the app version), it’s likely that an expressive TTS model will become available through their API in the future.